In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Input, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Creating batches for the datasets

In [ ]:
#find data sets. note these are not made public due to confidential nature.
train_path = "train_data/train"
valid_path = "train_data/valid"
test_path = "train_data/test"

In [ ]:
#generation of batches; batch_size can be adapted based on availability of GPU.
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=64)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=64)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=64, shuffle=False)

## Building the model

In [ ]:
#download the model, top is not included in order to do transfer learning.
resnet50_model = tf.keras.applications.resnet50.ResNet50(include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
resnet50_model.summary()

In [ ]:
#additional dense layers and dropout to successfully carry out transfer learning. Original output is flattened such that the original model functions as feature extractor.
top_layers = resnet50_model.output
top_layers = Flatten(name="top_flattening")(top_layers)
top_layers = Dense(units=1024,activation="ReLU",name="first_dense_top")(top_layers)
top_layers = Dropout(0.5,name="dropout_top")(top_layers)
top_layers = Dense(units=3, activation="softmax",name="linear_output")(top_layers)

In [ ]:
#the resnet model and the output layers are joined together
model = Model(inputs=resnet50_model.input, outputs=top_layers)

In [ ]:
#original model is set to be untrainable.
for layer in resnet50_model.layers:
	layer.trainable = False

In [ ]:
#the range of different learning rates that will be compared.
learning_rate = np.logspace(-4,-2,4)

## Printing the results

In [ ]:
#comparison of the learning rates and epochs necessary to compute them.
for lr in learning_rate:
  print("--------------------")
  print("learning rate = ", lr)
  print("--------------------")
  model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=5
  )